In [24]:
import rioxarray as rxr
import rasterio
import xarray as xr
import numpy as np

In [24]:
path = './../data/tif/liquid_water_2022-12-25.tif'
data = rxr.open_rasterio(path)
da = rasterio.open(path)

In [22]:
ds = xr.open_dataset('./../data/interp_nc/interp_liquid_water.nc')

In [25]:
lon = ds.longitude
lat = ds.latitude
lon_min, lon_max, lat_min, lat_max = lon.min(), lon.max(), lat.min(), lat.max()
res = (lon[1] - lon[0], lat[1] - lat[0])
# 创建GeoTransform对象,注意这个数据的lat是##从小到大##排序的
geotransform = (lon_min, res[0], 0, lat_max, 0, res[1])

In [27]:
def func_geo_to_imagexy(trans,lon,lat):
    '''
    根据GDAL六参数模型将地理坐标转换为图上坐标
    return:返回图上行列号(row,col)
    '''
    x,y=lon,lat
    a = np.array([[trans[1],trans[2]],[trans[4],trans[5]]])
    b = np.array([x-trans[0],y-trans[3]])
    col,row = np.linalg.solve(a,b).astype(int)
    return row,col
def func_imagexy_to_geo(trans,row,col):
    '''
    根据图上行列号转换GDAL六参数
    return:lon,lat
    '''
    lon = trans[0]+col*trans[1]+row*trans[2]
    lat = trans[3]+col*trans[4]+row*trans[5]
    return lon,lat

In [32]:
func_geo_to_imagexy(geotransform,108.3,30.5)

(2, 1)

In [29]:
ds

<xarray.Dataset>
Dimensions:    (time: 7, latitude: 135, longitude: 135)
Coordinates:
  * time       (time) datetime64[ns] 2022-12-25T05:00:00 ... 2022-12-31T05:00:00
  * latitude   (latitude) float64 30.63 30.58 30.53 30.48 ... 24.23 24.18 24.13
  * longitude  (longitude) float64 108.2 108.3 108.3 108.4 ... 114.7 114.7 114.8
Data variables:
    tclw       (time, latitude, longitude) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-02-16 08:18:00 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...